In [1]:
import requests
import json
import math

## Querying the API for bus locations and stops

In [2]:
start_time = 1539490442179
end_time   = 1539490499179
routes = ["14","14R","49"]

In [3]:
def list_string(arr):
    return f"{arr}".replace("'",'"')

In [4]:
query = f"""{{
  trynState(agency: "muni", startTime: "{start_time}", endTime: "{end_time}", routes: {list_string(routes)}) {{
    agency
    startTime
    routes {{
      rid
      stops {{
        sid
        name
        lat
        lon
      }}
      routeStates {{
        vtime
        vehicles {{
          vid
          lat
          lon
          heading
        }}
      }}
    }}
  }}
}}
""" # Braces need to be doubled for f-string

In [5]:
query_url = "https://06o8rkohub.execute-api.us-west-2.amazonaws.com/dev/graphql?query="+query
r = requests.get(query_url)

### Parsing the returned data

In [6]:
data = json.loads(r.text)
route_list = data['data']['trynState']['routes']

In [7]:
route_list[0]['routeStates'][1]['vehicles'][4]

{'heading': 165, 'lat': 37.755463, 'lon': -122.418823, 'vid': '7257'}

In [8]:
route_list[0]['stops'][8]

{'lat': 37.78075,
 'lon': -122.4090999,
 'name': 'Mission St & 6th St',
 'sid': '5538'}

## Function to find the closest stop to a bus

In [88]:
def get_dist(loc1,loc2,lat_scale=1,lon_scale=1):
    return abs(loc1['lat']-loc2['lat'])*lat_scale + abs(loc1['lon']-loc2['lon'])*lon_scale

In [89]:
def get_closest(loc,stop_list,lat_scale=1,lon_scale=1):
    # Running minimum distance
    running_min=500 #Dumb high number

    for stop in stop_list:
        current_min=get_dist(loc,stop,lat_scale,lon_scale)
        if current_min < running_min:
            running_min = current_min
            running_best_stop = stop
            
    return running_best_stop

In [84]:
# Scale latitude and longitude to miles
scale_miles =  {
    'lat_scale' : 69.0,
    'lon_scale' : 54.8
}
# math.cos(37.7*math.pi/180)

In [12]:
bus_1 = route_list[0]['routeStates'][1]['vehicles'][2]
stops_14 = route_list[0]['stops']

get_closest(bus_1, stops_14, lat_scale)

{'lat': 37.72635,
 'lon': -122.4336499,
 'name': 'Mission St & Francis St',
 'sid': '5590'}

In [13]:
route_list[0]['routeStates'][1]['vehicles'][2]['vid']

'7216'

### Closest stops for a single bus over time series

Robustness: Buses may not always have same index in `vehicles` array. Can query by `vid`, though it's more verbose.

In [83]:
[(int(x['vtime']), get_closest(x['vehicles'][0],stops_14)['name']) for x in route_list[0]['routeStates']]

[(1539490450077, 'Mission St & Excelsior Ave'),
 (1539490465083, 'Mission St & Excelsior Ave'),
 (1539490480087, 'Mission St & Silver Ave'),
 (1539490495092, 'Mission St & Silver Ave')]

In [74]:
[(int(x['vtime']), get_closest(x['vehicles'][13],stops_14)['name']) for x in route_list[0]['routeStates']]

[(1539490450077, 'Mission St & 4th St'),
 (1539490465083, 'Mission St & 4th St'),
 (1539490480087, 'Mission St & 4th St'),
 (1539490495092, 'Mission St & 3rd St')]

In [75]:
[(int(x['vtime']), get_closest(x['vehicles'][9],route_list[1]['stops'])['name']) for x in route_list[1]['routeStates']]

[(1539490450077, 'Mission St & 30th St'),
 (1539490465083, 'Mission St & Cortland Ave'),
 (1539490480087, 'Mission St & Cortland Ave'),
 (1539490495092, 'Mission St & Cortland Ave')]

Querying by `vid`, using comprehension syntax

In [76]:
[(int(x['vtime']), get_closest(next(y for y in x['vehicles'] if y['vid']=='6653'),route_list[1]['stops'])['name']) for x in route_list[1]['routeStates']]

[(1539490450077, 'Mission St & 30th St'),
 (1539490465083, 'Mission St & Cortland Ave'),
 (1539490480087, 'Mission St & Cortland Ave'),
 (1539490495092, 'Mission St & Cortland Ave')]

We can also compute the closest bus to a given stop.

In [77]:
[get_closest(stops_14[7], x['vehicles'])['vid'] for x in route_list[0]['routeStates']]

['7227', '7227', '7227', '7202']

And distance to closest bus / stop in miles.

In [96]:
(lambda x: (get_dist(x,bus_1,**scale_miles),x['name']))(get_closest(bus_1,stops_14))

(0.01752807999944537, 'Mission St & Francis St')

In [105]:
(lambda x: (get_dist(x,stops_14[7],**scale_miles),x['vid']))(get_closest(stops_14[7],route_list[0]['routeStates'][3]['vehicles']))

(0.4950115000003563, '7202')